In [81]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
import json
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import SmoothingFunction
import nltk
import os
import seaborn as sns

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
file_path = "output/devset/devset_with_contexts.parquet.gzip"
with open(file_path, "rb") as f:
    dev_set = pd.read_parquet(f)

In [3]:
dev_set

,question,answer,text,pnumber,law number,tfidf_context,bm25_context,bert_cls_context,bert_max_context,bert_mean_context
0,"Hvad har ejeren af en ejerlejlighed, sammen me...","Grunden, fælles bestanddele og tilbehør.",Ejeren af en ejerlejlighed har sammen med andr...,3,LOV nr 908 af 18/06/2020,[Ejeren af en ejerlejlighed har sammen med and...,[Ejeren af en ejerlejlighed har sammen med and...,[Hver ejerlejlighed anses som en selvstændig f...,"[Det kan aftales, at lejeren af et blandet lej...",[Ved skriftlig budgivning opfordrer fogeden sk...
1,Hvem fastsætter eller aftaler bestemmelser om ...,Finansministeren fastsætter eller aftaler best...,Højskolen skal følge de af finansministeren fa...,30,LBK nr 780 af 08/08/2019,[Højskolen skal følge de af finansministeren f...,[Højskolen skal følge de af finansministeren f...,[Professionshøjskolens navn fastsættes i vedtæ...,[Virksomheden kan efter finansministerens nærm...,[Told- og skatteforvaltningen kan fastsætte re...
2,Hvad skal Beskæftigelsesministeriet og Finanst...,Den indsendte årsrapport skal i det mindste in...,Uden ugrundet ophold efter repræsentantskabets...,25 l,LBK nr 1110 af 10/10/2014,[Uden ugrundet ophold efter bestyrelsens godke...,[Uden ugrundet ophold efter bestyrelsens godke...,[Arbejdsmarkedets Erhvervssikring betaler afgi...,[Den godkendte årsrapport skal indsendes til E...,"[§ 10 finder ikke anvendelse for gaver, der er..."
3,Hvor mange procent må kapitalandele i og lån y...,Kapitalandele i og lån ydet til en virksomhed ...,Følgende grænser for Arbejdsmarkedets Tillægsp...,26 e,LBK nr 1110 af 10/10/2014,[Følgende grænser for Lønmodtagernes Dyrtidsfo...,[Følgende grænser for Lønmodtagernes Dyrtidsfo...,[Et pengeinstituts kapitalandele erhvervet for...,[Bestyrelsen i et e-pengeinstitut eller betali...,[Skadesforsikringsselskaberne skal efter hvert...
4,Hvad er en betingelse for retten til jobpræmie?,Det er en betingelse for retten til jobpræmie ...,"Det er en betingelse for retten til jobpræmie,...",9,LOV nr 287 af 29/03/2017,[Det er en betingelse for retten til jobpræmie...,[Det er en betingelse for retten til jobpræmie...,[Det er en betingelse for retten til jobpræmie...,[Beskæftigelsesministeren kan fastsætte budget...,"[Det påhviler anklagemyndigheden at påse, at e..."
...,...,...,...,...,...,...,...,...,...,...
98,Hvordan anføres kandidatlister på stemmesedler?,I særskilte felter.,Kandidatlisterne anføres på stemmesedlen i sær...,46,LBK nr 6 af 08/01/2024,[Valgbestyrelsen udarbejder stemmesedler på gr...,[Valgbestyrelsen udarbejder stemmesedler på gr...,[Kandidatlisterne anføres på stemmesedlen i sæ...,[Kulturministeren kan fastsætte nærmere regler...,"[Ministeren for fødevarer, landbrug og fiskeri..."
99,Hvem iværksætter beslaglæggelse?,Politiet.,Politiet iværksætter beslaglæggelse. Politiet ...,807,LBK nr 250 af 04/03/2024,"[Beløb, som en person har til gode hos en virk...","[Beløb, som en person har til gode hos en virk...","[Retsformanden leder forhandlingen., Arbejdsti...",[Studievalg Danmark kan drive indtægtsdækket v...,[Regionsrådet og kommunalbestyrelsen koordiner...
100,Hvis interesser skal foranstaltninger mod inte...,De forvaltede alternative investeringsfondes e...,En forvalter af alternative investeringsfonde ...,23,LBK nr 231 af 01/03/2024,[Delegation eller videredelegation af porteføl...,[Delegation eller videredelegation af porteføl...,"[Hvis en afdeling er bevisudstedende, skal den...",[Bestyrelsen i et e-pengeinstitut eller betali...,[Erhvervsstyrelsen kan foretage udgående kontr...
101,Hvad skal valgstyrere eller tilforordnede vælg...,At stemmekasserne er tomme.,Afstemningen begynder kl. Inden stemmeafgivnin...,38,LBK nr 1432 af 01/12/2023,[Afstemningen begynder kl. Inden stemmeafgivni...,[Afstemningen begynder kl. Inden stemmeafgivni...,"[Valgbestyrelsen underretter de valgte om, at ...",[Afstemningen begynder kl. Inden stemmeafgivni...,[Medmindre strengere straf er forskyldt efter ...


#### Functions

In [ ]:
def replace_nbsp(obj):
    """
    Recursively replaces non-breaking space characters (\xa0) with regular spaces
    in strings within nested data structures.
    
    Args:
        obj: Input object which can be a dictionary, list, string or other type
        
    Returns:
        Object of same structure as input but with \xa0 replaced with spaces in all strings
    """
    if isinstance(obj, dict):
        return {k: replace_nbsp(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_nbsp(i) for i in obj]
    elif isinstance(obj, str):
        return obj.replace('\xa0', ' ')
    else:
        return obj

def generate_paragraphs(input_file):
    """
    Generates a list of dictionaries containing paragraph information from legal documents.
    
    Args:
        input_file: Path to input JSON file containing legal document data
        output_file: Path to output file to write the processed data
        
    Returns:
        rag_list: List of dictionaries, where each dictionary contains:
            - paragraf_nr: Paragraph number
            - lovnavn: Name of the law
            - text: Combined text of all subsections in the paragraph
    """
    with open(input_file) as f:
        retsinfo = json.load(f)

    paragraphs = []
    idx = 0
    for lov in retsinfo:
        for kapitel in lov['kapitler']:
            lov_navn = lov['shortName']
            for paragraffer in kapitel['paragraffer']:
                temp_paragraf_dict = {}
                temp_paragraf_dict['paragraf_nr'] = paragraffer['nummer']
                temp_paragraf_dict['lovnavn'] = lov_navn
                temp_paragraf_list = []
                for styk in paragraffer['stk']:
                    temp_paragraf_list.append(styk['tekst'])
                temp_paragraf_dict['text'] = ' '.join(temp_paragraf_list)
                paragraphs.append(temp_paragraf_dict)

    paragraphs = replace_nbsp(paragraphs)
    return paragraphs

#### Count tokens in corpus

In [ ]:
model_answers = {}
for file in os.listdir('output/inference'):
        if file.endswith('.txt'):
            model_name = file.replace(".txt", "")
            with open(f"output/inference/{file}", "r") as f:
                model_answers[model_name] = f.read().splitlines()
                print(f"Loaded answers from model: {model_name}")

Loaded answers from model: neo_gen_bert_max_k2
Loaded answers from model: t5_gen_bert_mean_k1
Loaded answers from model: t5_gen_random_context
Loaded answers from model: neo_gen_bert_max_k3
Loaded answers from model: neo_gen_bert_max_k1
Loaded answers from model: t5_gen_bert_mean_k3
Loaded answers from model: t5_gen_bert_mean_k2
Loaded answers from model: t5_gen_bert_cls_k3
Loaded answers from model: t5_gen_bert_cls_k2
Loaded answers from model: t5_gen_bert_cls_k1
Loaded answers from model: neo_gen_random_context
Loaded answers from model: neo_gen_upper_bound
Loaded answers from model: t5_gen_tfidf_k2
Loaded answers from model: t5_gen_tfidf_k3
Loaded answers from model: neo_gen_baseline
Loaded answers from model: t5_gen_tfidf_k1
Loaded answers from model: t5_gen_bert_max_k1
Loaded answers from model: t5_gen_bm25_k2
Loaded answers from model: t5_gen_bm25_k3
Loaded answers from model: t5_gen_bert_max_k2
Loaded answers from model: t5_gen_bm25_k1
Loaded answers from model: t5_gen_bert_max_

In [36]:
model_answers['neo_gen_baseline']

['l: Hvad er forskellen på en ejerlejlighed og en andelsbolig? Indsæt svar her: Spørgsmål: Hvad er forskellen på en ejerlejlighed og en andelsbolig? Indsæt svar her: Spørgsmål: Hvad er forskellen på en ejerlejlighed og en andelsbolig? Indsæt svar her: Spørgsmål: Hvad er forskellen på en ejerlejlighed og en andelsbolig? Indsæt svar her: Spørgsmål: Hvad er forskellen på en ejerlejlighed og en andelsbolig? Indsæt svar her: Spørgsmål: Hvad er forskellen på en ejerlejlighed og en andelsbolig? Indsæt sva',
 'jskolen er en selvejende institution, der er oprettet ved lov af den daværende regering. Højskolen er en selvejende institution, der er oprettet ved lov af den daværende regering. Højskolen er en selvejende institution, der er oprettet ved lov af den daværende regering. Højskolen er en selvejende institution, der er oprettet ved lov af den daværende regering. Højskolen er en selvejende institution, der er oprettet ved lov af den daværende regering. Højskolen er en selvejende institution,

In [66]:
neo_tokenizer = AutoTokenizer.from_pretrained('KennethTM/gpt-neo-1.3B-danish')
t5_tokenizer = AutoTokenizer.from_pretrained('strombergnlp/dant5-large')

# Creating a dictionary with mean output lengths for all the models
mean_len_dict = {}
for key in tqdm(model_answers):
    if str(key)[:3] == 'neo':
        temp_list = []
        for answer in model_answers[key]:
            temp_list.append(len(neo_tokenizer.tokenize(answer)))
        mean_len_dict[key] = np.mean(temp_list)
    elif str(key[:2]) == 't5':
        temp_list = []
        for answer in model_answers[key]:
            temp_list.append(len(t5_tokenizer.tokenize(answer)))
        mean_len_dict[key] = np.mean(temp_list)
    else:
        print('fail')

# Sorting keys 
myKeys = list(mean_len_dict.keys())
myKeys.sort()

# Sorted Dictionary
mean_len_dict = {i: mean_len_dict[i] for i in myKeys}

100%|██████████| 36/36 [00:00<00:00, 63.80it/s]


In [79]:
neo_k1 = {}
neo_k2 = {}
neo_k3 = {}

t5_k1 = {}
t5_k2 = {}
t5_k3 = {}


for key in mean_len_dict.keys():
    if str(key)[:3] == 'neo':
        if str(key[-2:]) == 'k1':
            neo_k1[key] = mean_len_dict[key]
        elif str(key[-2:]) == 'k2':
            neo_k2[key] = mean_len_dict[key]
        elif str(key[-2:]) == 'k3':
            neo_k3[key] = mean_len_dict[key]
        else:
            print('pass')
    elif str(key[:2]) == 't5':
        if str(key[-2:]) == 'k1':
            t5_k1[key] = mean_len_dict[key]
        elif str(key[-2:]) == 'k2':
            t5_k2[key] = mean_len_dict[key]
        elif str(key[-2:]) == 'k3':
            t5_k3[key] = mean_len_dict[key]
        else:
            print('pass')

pass
pass
pass
pass
pass
pass


In [80]:
neo_k1

{'neo_gen_bert_cls_k1': 45.00970873786408,
 'neo_gen_bert_max_k1': 45.22330097087379,
 'neo_gen_bert_mean_k1': 73.88349514563107,
 'neo_gen_bm25_k1': 77.37864077669903,
 'neo_gen_tfidf_k1': 68.7378640776699}

In [67]:
keys = list(my_dict.keys())
# get values in the same order as keys, and parse percentage values
vals = [float(my_dict[k][:-1]) for k in keys]
sns.barplot(x=keys, y=vals)

{'neo_gen_baseline': 94.79611650485437,
 'neo_gen_bert_cls_k1': 45.00970873786408,
 'neo_gen_bert_cls_k2': 63.85436893203884,
 'neo_gen_bert_cls_k3': 66.28155339805825,
 'neo_gen_bert_max_k1': 45.22330097087379,
 'neo_gen_bert_max_k2': 65.75728155339806,
 'neo_gen_bert_max_k3': 69.36893203883496,
 'neo_gen_bert_mean_k1': 73.88349514563107,
 'neo_gen_bert_mean_k2': 85.23300970873787,
 'neo_gen_bert_mean_k3': 86.44660194174757,
 'neo_gen_bm25_k1': 77.37864077669903,
 'neo_gen_bm25_k2': 91.98058252427184,
 'neo_gen_bm25_k3': 90.6504854368932,
 'neo_gen_random_context': 81.02912621359224,
 'neo_gen_tfidf_k1': 68.7378640776699,
 'neo_gen_tfidf_k2': 88.28155339805825,
 'neo_gen_tfidf_k3': 87.4368932038835,
 'neo_gen_upper_bound': 73.55339805825243,
 't5_gen_baseline': 86.9126213592233,
 't5_gen_bert_cls_k1': 84.8252427184466,
 't5_gen_bert_cls_k2': 34.48543689320388,
 't5_gen_bert_cls_k3': 36.728155339805824,
 't5_gen_bert_max_k1': 75.02912621359224,
 't5_gen_bert_max_k2': 39.45631067961165,

In [12]:
paragraphs = generate_paragraphs('/Users/adamwagnerhoegh/ANLPDP_exam/data/domsdatabasen.retsinformation_newer.json')

In [22]:
len(tokenizer.tokenize(paragraphs[4]['text']))

74

In [24]:
# load model and tokenizer and set device
neo_tokenizer = AutoTokenizer.from_pretrained('KennethTM/gpt-neo-1.3B-danish')
t5_tokenizer = AutoTokenizer.from_pretrained('strombergnlp/dant5-large')

neo_p_lens = []
t5_p_lens = []

for item in tqdm(paragraphs):
    neo_p_lens.append(len(neo_tokenizer.tokenize(item['text'])))
    t5_p_lens.append(len(t5_tokenizer.tokenize(item['text'])))

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
  0%|          | 0/42593 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Token indices sequence length is longer than the specified maximum sequence length for this model (1178 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 42593/42593 [00:23<00

In [26]:
print(f'Mean token length of paragraph (neo): {np.mean(neo_p_lens)}')
print(f'Mean token length of paragraph (t5): {np.mean(t5_p_lens)}')


Mean token length of paragraph (neo): 132.69074730589534
Mean token length of paragraph (t5): 128.02904233089944


In [28]:
print(f'Token length of corpus (neo): {sum(neo_p_lens)}')
print(f'Token length of corpus (t5): {sum(t5_p_lens)}')

Token length of corpus (neo): 5651697
Token length of corpus (t5): 5453141


#### Calculate mean token length of outputs

In [6]:
cls_embeddings = torch.load('/Users/adamwagnerhoegh/ANLPDP_exam/output/embeddings/cls_embeddings_DanskBERT.pt')

In [7]:
cls_embeddings.size()

torch.Size([42593, 768])

In [8]:
torch.mean(torch.std(cls_embeddings, dim=1))

tensor(0.9152)

In [10]:
paragraphs = generate_paragraphs('/Users/adamwagnerhoegh/ANLPDP_exam/data/domsdatabasen.retsinformation_newer.json')

In [11]:
def random_dense_retrieval(embeddings, corpus, pooling='cls', k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    pooling = 'cls', 'max' or 'mean'
    """
    
    embedding_vector = torch.rand(1, 768)

    # normalise embeddings (to get cosine similarity from dot product)
    embedding_vector_normalised = embedding_vector / torch.norm(embedding_vector, dim=-1, keepdim=True)
    embeddings_matrix_normalised = embeddings / torch.norm(embeddings, dim=-1, keepdim=True)

    # get paragraphs with highest similarity scores
    dense_retrieval = embeddings_matrix_normalised @ torch.transpose(embedding_vector_normalised, 0, 1)
    sorted_retrieval = torch.sort(dense_retrieval, descending=True, stable=True, dim=0)
    fixed_retrieval_list = [(item, idx) for (item, idx) in zip(sorted_retrieval[0], sorted_retrieval[1]) if torch.isnan(item) == False]
    top_k_indices = [item[1] for item in fixed_retrieval_list[:k]]
    document = [corpus[i]['text'] for i in top_k_indices]

    return document

In [30]:
random_context = []

for _ in tqdm(dev_set['question']):
    random_context.append(random_dense_retrieval(cls_embeddings, paragraphs))

100%|██████████| 103/103 [00:19<00:00,  5.23it/s]


In [32]:
with open("random_context.json", "w") as save:
	json.dump(random_context, save)

In [24]:
with open('output/inference/neo_gen_random_context.txt', 'r') as file:
    # Read all lines into a list
    random_answers = file.readlines()

In [32]:
with open('output/inference/neo_gen_bert_cls_k1.txt', 'r') as file:
    # Read all lines into a list
    neo_k1_answer = file.readlines()

In [25]:
random_answers

['En ejerlejlighed er en selvstændig fast ejendom, som ejes af ejeren af en ejerlejlighed. En ejerlejlighed er en selvstændig fast ejendom, som ejes af ejeren af en ejerlejlighed. En ejerlejlighed er en selvstændig fast ejendom, som ejes af ejeren af en ejerlejlighed. En ejerlejlighed er en selvstændig fast ejendom, som ejes af ejeren af en ejerlejlighed. En ejerlejlighed er en selvstændig fast ejendom, som ejes af ejeren af en ejerlejlighed. En ejerlejlighed er en selvstændig fast ejendom, som ejes af ejeren af en ejerlejlighed. En ejerlejlighed er en\n',
 'Indfødsret meddeles til følgende personer, som er statsborgere i andre vestlige lande end lande i Norden: Wiebke Annette Mundt, Wiebke Annette Mundt, Daniel Antal-Paulauskas, Miriam Bang, Richard Anders Batchelor, Luna Bongart, Leon Bossen, Michael Edward Brewer, Aidas Buemann, Frank-Martin Baasch, Karen Diederichsen, Jan Einfeldt, Kay-Hermann Einfeldt, Tanja En\n',
 '2) Beskæftigelsesministeriets bekendtgørelse nr. 810 af 28. juni

In [19]:
def calculate_bleu(answers, gold_answers):
    scores = []
    for answer, gold_answer in zip(answers, gold_answers):
        score = sentence_bleu([gold_answer], answer, smoothing_function=SmoothingFunction().method1)
        scores.append(score)
    return scores

def calculate_rouge(answers, gold_answers):
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    for answer, gold_answer in zip(answers, gold_answers):
        scores = scorer.score(answer, gold_answer)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
        
    return rouge1_scores, rouge2_scores, rougeL_scores

def calculate_meteor(answers, gold_answers):
    scores = []
    for answer, gold_answer in zip(answers, gold_answers):
        answer_tokens = answer.split()
        gold_answer_tokens = gold_answer.split()
        score = meteor_score([gold_answer_tokens], answer_tokens, gamma=0.0) # set gamma (word order penalty) to 0 to avoid bug where two identical sentences are not scored 1
        scores.append(score)
    return scores


In [30]:
print(f"BLEU: {np.mean(calculate_bleu(random_answers, dev_set['answer']))}")
print(f"ROUGE-L: {np.mean(calculate_rouge(random_answers, dev_set['answer'])[2])}")
print(f"METEOR: {np.mean(calculate_meteor(random_answers, dev_set['answer']))}")


BLEU: 0.06535894659682062
ROUGE-L: 0.044419446900313526
METEOR: 0.07943291113182534


In [33]:
print(f"BLEU: {np.mean(calculate_bleu(neo_k1_answer, dev_set['answer']))}")
print(f"ROUGE-L: {np.mean(calculate_rouge(neo_k1_answer, dev_set['answer'])[2])}")
print(f"METEOR: {np.mean(calculate_meteor(neo_k1_answer, dev_set['answer']))}")

BLEU: 0.1122419548289757
ROUGE-L: 0.10222230224439338
METEOR: 0.14409572606976204


In [4]:
def answer_len(path):
    with open(path, 'r') as file:
    # Read all lines into a list
        answers = file.readlines()

    return np.mean([len(item) for item in answers])


In [11]:
print('Neo CLS')
print(answer_len('output/inference/neo_gen_bert_cls_k1.txt'))
print(answer_len('output/inference/neo_gen_bert_cls_k2.txt'))
print(answer_len('output/inference/neo_gen_bert_cls_k3.txt'))

Neo CLS
216.29126213592232
340.15533980582524
359.3980582524272


In [12]:
print('Neo max')
print(answer_len('output/inference/neo_gen_bert_max_k1.txt'))
print(answer_len('output/inference/neo_gen_bert_max_k2.txt'))
print(answer_len('output/inference/neo_gen_bert_max_k3.txt'))

Neo max
227.0
357.4757281553398
381.1456310679612


In [13]:
print('Neo mean')
print(answer_len('output/inference/neo_gen_bert_mean_k1.txt'))
print(answer_len('output/inference/neo_gen_bert_mean_k2.txt'))
print(answer_len('output/inference/neo_gen_bert_mean_k3.txt'))

Neo mean
339.7864077669903
386.15533980582524
410.873786407767


In [16]:
print('Neo BM25')
print(answer_len('output/inference/neo_gen_bm25_k1.txt'))
print(answer_len('output/inference/neo_gen_bm25_k2.txt'))
print(answer_len('output/inference/neo_gen_bm25_k3.txt'))

Neo BM25
389.0388349514563
482.0194174757282
481.4466019417476


In [18]:
print('Neo TF-IDF')
print(answer_len('output/inference/neo_gen_tfidf_k1.txt'))
print(answer_len('output/inference/neo_gen_tfidf_k2.txt'))
print(answer_len('output/inference/neo_gen_tfidf_k3.txt'))

Neo TF-IDF
339.41747572815535
447.29126213592235
446.8349514563107
